In [ ]:
import pandas as pd
import xgboost
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, recall_score, precision_score, accuracy_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
import neurokit2 as nk
import hrvanalysis as hrv
from scipy import ndimage
from sklearn.svm import SVC
from sklearn.datasets import make_classification
import numpy as np
from sklearn.feature_selection import RFECV
import os

In [ ]:
# Data preprocessing and feature extraction

folder_path = '/home/mw/input/hfindex7735/vol1/PPC'
npz_files = [os.path.join(root, file) for root, _, files in os.walk(folder_path) for file in files if file.endswith('.npz')]
ecg = []
rsp_ch = []
rsp_ab = []
spo = []
acc = []
for i, file_path in enumerate(npz_files):
    data = np.load(file_path)
    ecg.append(data['ecg_list'])
    rsp_ch.append(data['rsp_ch_list'])
    rsp_ab.append(data['rsp_ab_list'])
    spo.append(data['spo_value'])
    acc.append(data['acc'])


ecg_raw = ecg
rsp_ch_raw = rsp_ch
rsp_ab_raw = rsp_ab
spo_raw = spo
acc_raw = acc

def smooth_signal(signal, size):
    smoothed_signal = np.zeros(len(signal))
    for i in np.arange(len(signal)):
        if i == 0:
            smoothed_signal[i] = signal[i]
        elif i < np.divide((size - 1), 2):
            smoothed_signal[i] = np.mean(signal[:i * 2 + 1])
        elif i > len(signal) - np.divide((size - 1), 2) - 1:
            smoothed_signal[i] = np.mean(signal[i - (len(signal) - 1 - i):len(signal)])
        else:
            start = int(i - np.divide((size - 1), 2))
            end = int(i + 1 + np.divide((size - 1), 2))
            smoothed_signal[i] = np.mean(signal[start:end])
    return smoothed_signal


def remove_outliers(signal, size):

    for i in np.arange(size, len(signal) - size):
        mean = np.mean(np.hstack((signal[i-size:i],signal[i+1:i+size+1])))
        std = np.std(np.hstack((signal[i-size:i],signal[i+1:i+size+1])))
        if ((signal[i] > mean + 3*std) | (signal[i] < mean - 3*std)):
            signal[i] = mean
    return signal


def spo_process(spo2_values):
    spo2_values = np.array(spo2_values)
    spo2_values = spo2_values[spo2_values != 0]
    
    percent_above_95 = np.sum(spo2_values > 95) / len(spo2_values) * 100
    percent_above_90 = np.sum(spo2_values > 90) / len(spo2_values) * 100
    percent_above_85 = np.sum(spo2_values > 85) / len(spo2_values) * 100
    percent_above_80 = np.sum(spo2_values > 80) / len(spo2_values) * 100
    
    return {
        "percent_above_95": percent_above_95,
        "percent_above_90": percent_above_90,
        "percent_above_85": percent_above_85,
        "percent_above_80": percent_above_80
    }

def rr_process(ecg, ecg_peaks, th=10):
    """
    Calculate the arrhythmic load.
    :param ecg: An array of electrocardiogram signals
    :param ecg_peaks: Indices of R-wave positions
    :param th: Threshold for heart rate variation, default is 10 beats per minute
    :return: Arrhythmic load (number of arrhythmias per total time)
    """
    ect_r_ind = []  # List to store arrhythmic positions
    
    total_time_seconds = len(ecg) / 200
    total_time = total_time_seconds / 60
    
    rr_intervals = np.diff(ecg_peaks) / 200
    hr_per_peak = 60 / rr_intervals
    
    hr_per_peak = hr_per_peak.tolist()  # Convert heart rate list to standard list format
    ecg_peaks = ecg_peaks[1:].tolist()  # Remove the first R-wave since it has no previous comparison
    et_count = 0  # Arrhythmia count
    i = 1  # Start checking from the second position

    # Check heart rate differences before and after each R-wave position
    while i < len(hr_per_peak) - 2:
        # If the heart rate variation exceeds the threshold, consider it an arrhythmia
        if (abs(hr_per_peak[i] - hr_per_peak[i - 1]) >= th and abs(hr_per_peak[i] - hr_per_peak[i + 1]) >= th) or \
           abs(hr_per_peak[i + 1] - hr_per_peak[i + 2]) >= th:
            if abs(hr_per_peak[i] - hr_per_peak[i - 1]) < th:
                i += 1
            else:
                hr_per_peak.pop(i)
                ecg_peaks.pop(i)
                ect_r_ind.append(ecg_peaks[i])  # Record the arrhythmic position
                et_count += 1  # Increase the arrhythmia count
        else:
            i += 1  # If no arrhythmia, continue to the next position

    # Calculate arrhythmic load
    ectopic_load = et_count / total_time if total_time > 0 else 0  # Avoid division by zero

    return ectopic_load


def rsp_process(rsp_info,respList):
    rip_peaks = rsp_info['RSP_Peaks']
    rip_troughs = rsp_info['RSP_Troughs']
    if rip_peaks[0] < rip_troughs[0]:
        rip_peaks = np.delete(rip_peaks, 0)
    if rip_peaks[-1] > rip_troughs[-1]:
        rip_peaks = np.delete(rip_peaks, -1)
    resp_feature = dict()
    br = 60/(np.diff(rip_troughs)/ 25)
    br_index = np.arange(len(br))
    resp_feature['br_mean'] = np.mean(br[br_index])
    resp_feature['br_max'] = np.max(br[br_index])
    resp_feature['br_min'] = np.min(br[br_index])
    resp_feature['br_cv'] = np.std(br[br_index]) / np.mean(br[br_index]) * 100
    
    TI_interval, TE_interval, VT_in, VT_ex = [], [], [], []

    for i in range(len(rip_peaks)):
        temp = rip_troughs[i + 1] - rip_peaks[i]  # huqi
        temp2 = rip_peaks[i] - rip_troughs[i]  # xiqi
        vt_ex_temp = respList[rip_peaks[i]] - respList[rip_troughs[i + 1]]
        vt_in_temp = respList[rip_peaks[i]] - respList[rip_troughs[i]]
        TI_interval.append(temp2)
        TE_interval.append(temp)
        VT_ex.append(vt_ex_temp)
        VT_in.append(vt_in_temp)

    TI_interval = np.array(TI_interval) / 25
    TE_interval = np.array(TE_interval) / 25
    TI_ratio = TI_interval / (TI_interval + TE_interval)
    resp_feature['TI_mean'] = np.mean(TI_interval[br_index])
    resp_feature['TI_std'] = np.std(TI_interval[br_index])
    resp_feature['TI_cv'] = (np.std(TI_interval[br_index])) / (np.mean(TI_interval[br_index])) * 100
    resp_feature['TE_mean'] = np.mean(TE_interval[br_index])
    resp_feature['TE_std'] = np.std(TE_interval[br_index])
    resp_feature['TE_cv'] = (np.std(TE_interval[br_index])) / (np.mean(TE_interval[br_index])) * 100
    resp_feature['TI_ratio_mean'] = np.mean(TI_ratio[br_index])
    resp_feature['TI_ratio_std'] = np.std(TI_ratio[br_index])
    resp_feature['TI_ratio_cv'] = (np.std(TI_ratio[br_index])) / (np.mean(TI_ratio[br_index])) * 100
   
    min_ven_in = br * VT_in
    min_ven_ex = br * VT_ex
    
    resp_feature['min_ven_in_mean'] = np.mean(min_ven_in[br_index])
    resp_feature['min_ven_in_std'] = np.std(min_ven_in[br_index])
    resp_feature['min_ven_in_cv'] = (np.std(min_ven_in[br_index])) / (np.mean(min_ven_in[br_index])) * 100
    resp_feature['min_ven_ex_mean'] = np.mean(min_ven_ex[br_index])
    resp_feature['min_ven_ex_std'] = np.std(min_ven_ex[br_index])
    resp_feature['min_ven_ex_cv'] = (np.std(min_ven_ex[br_index])) / (np.mean(min_ven_ex[br_index])) * 100
    
    RSBI_in = br/ VT_in
    RSBI_ex = br/ VT_in
    
    resp_feature['RSBI_in_mean'] = np.mean(RSBI_in[br_index])
    resp_feature['RSBI_in_std'] = np.std(RSBI_in[br_index])
    resp_feature['RSBI_in_cv'] = (np.std(RSBI_in[br_index])) / (np.mean(RSBI_in[br_index])) * 100
    resp_feature['RSBI_ex_mean'] = np.mean(RSBI_ex[br_index])
    resp_feature['RSBI_ex_std'] = np.std(RSBI_ex[br_index])
    resp_feature['RSBI_ex_cv'] = (np.std(RSBI_ex[br_index])) / (np.mean(RSBI_ex[br_index])) * 100
    
    return rsp_feature


def SleepApneaDetector():
    """
    Pseudocode for SleepApneaDetector:
	Input:
	    rsp_ch: An array of chest respiratory signals 
	    rsp_ab: An array of abdominal respiratory signals 
	    spo: An array of blood oxygen saturation signals
	    acc: An array of three-axis acceleration signals
	    
	Output:
	    ahi: Apnea-Hypopnea Index (AHI)

	Procedure:
    ### 1. Data Preparation
    1. **Input Data**: Obtain respiratory signals, blood oxygen saturation data, and triaxial acceleration data. 
    2. **Initialize Parameters**: Set the calculation parameters for the apnea threshold (TH_A) and hypopnea threshold (TH_H) (e.g., TH_A = AMP_MAX × 0.35, TH_H = AMP_MAX × 0.7, with AMP_MAX initially set to 0) for subsequent threshold calculations; set the time window for AHI calculation (e.g., 1 hour, adjustable according to actual needs) and event counters (e.g., apnea_count = 0, hypopnea_count = 0, used to record the number of apnea and hypopnea events respectively).
    
    ### 2. Tidal Volume Calculation and Preprocessing
    1. Calculate the tidal volume (VT) using the formula \(VT = K × RC + M × AB\) based on the preprocessed chest and abdominal respiratory signals, where K is the chest respiratory signal fitting coefficient, M is the abdominal respiratory signal fitting coefficient, RC is the preprocessed chest respiratory signal, and AB is the preprocessed abdominal respiratory signal.
    2. Filter and standardize the calculated tidal volume.
    
    ### 3. Calculate Respiratory Amplitude and Respiratory Amplitude Baseline
    1. Detect the peaks and valleys of the tidal volume to calculate the respiratory amplitude (AMP) sequence (the amplitude of the peak signal minus the amplitude of the adjacent valley signal). If \(AMP ≤ 0.15\), it is considered a false peak and removed.
    2. Calculate the respiratory amplitude baseline (AMP_MAX) by calculating the obtained AMP.
    3. Interpolate the respiratory amplitude and respiratory amplitude baseline according to the sampling frequency of the chest and abdominal respiratory signals to make them correspond to the respiratory signal.
    
    ### 4. Calculate Apnea and Hypopnea Thresholds
    1. Calculate the apnea threshold (TH_A) using the formula \(TH_A = AMP_MAX × 0.35\) based on the currently calculated AMP_MAX.
    2. Calculate the hypopnea threshold (TH_H) using the formula \(TH_H = AMP_MAX × 0.7\).
    
    ### 5. Preliminary Determination of Sleep Respiratory Events
    1. Based on the detection results of the tidal volume peaks and valleys, preliminarily judge the sleep respiratory events according to the following criteria:
        - **Central Sleep Apnea Judgment**:
            - If the interval between two adjacent peaks of the respiratory amplitude (AMP) curve is greater than 10 seconds, and there is no respiratory effort signal in both chest and abdominal respirations, it is judged as central sleep apnea, and apnea_count is incremented by 1.
            - If the AMP is less than TH_A for more than 10 seconds, and no peak is detected, or the peak interval exceeds 10 seconds, it is judged as central sleep apnea, and apnea_count is incremented by 1.
        - **Obstructive Sleep Apnea Judgment**:
            - If the interval between two adjacent peaks of the respiratory amplitude (AMP) curve is greater than 10 seconds, and at least one of the chest and abdominal respiratory signals shows respiratory effort, it is judged as obstructive sleep apnea, and apnea_count is incremented by 1.
            - If the AMP is less than TH_A for more than 10 seconds, and more than 6 peaks are detected, or the detected peaks are no more than 6 and the peak interval does not exceed 10 seconds, it is judged as obstructive sleep apnea, and apnea_count is incremented by 1.
        - **Mixed Sleep Apnea Judgment**: If the same time period simultaneously meets the judgment criteria of central sleep apnea and obstructive sleep apnea, and the central apnea occurs before the obstructive apnea, it is judged as mixed sleep apnea, and apnea_count is incremented by 1 (here, it can be considered whether to count the mixed sleep apnea separately or include it in the total number of apnea events according to specific requirements. Assume it is included in apnea_count).
        - **Hypopnea Judgment**: If the AMP is less than TH_H but greater than TH_A for more than 10 seconds, and accompanied by a decrease in blood oxygen saturation of more than 4%, it is judged as hypopnea, and hypopnea_count is incremented by 1.
    2. For each detected event, record its occurrence time (start_time) and duration (duration).
    
    ### 6. Calibrate Sleep Respiratory Events
    1. **Delete False Positives Caused by Artifacts**:
        - Calculate the signal quality index, which is the variance of the respiratory amplitude change (VAR(diff(AMP))), with the variance window set to 10 breaths. If there is a triaxial acceleration signal, the threshold is set to 4 times AMP_MAX; if there is no triaxial acceleration signal, the threshold is set to 2 times AMP_MAX. If the signal quality is poor, remove the segment of the signal and the previously detected apnea events within 2 minutes after this segment (reduce apnea_count or hypopnea_count accordingly).
        - Calculate the mean respiratory amplitude Mean(AMP) of each apnea and hypopnea event, and the mean respiratory amplitude Mean(AMP_after2min) 2 minutes after the event. If \(Mean(AMP) >= 0.7 * Mean(AMP_after2min)\), it is considered interference and the event is removed (reduce the corresponding count).
    2. **Calibrate with Blood Oxygen Signal**:
        - Calibrate the results after the previous step with the blood oxygen signal. If the blood oxygen does not decrease by 4% or more after a hypopnea event, remove the hypopnea event (hypopnea_count is decremented by 1); if the blood oxygen does not decrease by 3% or more after an apnea event, remove the apnea event (apnea_count is decremented by 1). During calibration, record the position of blood oxygen loss. If there is a loss of blood oxygen for more than 10 seconds when a respiratory event occurs, retain the event.
    3. **Calibrate with Triaxial Acceleration Signal**: Use the triaxial acceleration signal to identify the subject's sleeping position and activity state, and remove the apnea events detected during non-lying and active states and within 1 minute after that (reduce apnea_count accordingly).
    
    ### 7. Calculate AHI
    1. When the set time window (e.g., 1 hour) is reached, calculate the AHI using the formula \(AHI = (apnea_count + hypopnea_count) / time window (in hours)\) based on the current number of apnea events (apnea_count) and hypopnea events (hypopnea_count).
    2. Output the calculated AHI value for evaluating the severity of the subject's sleep apnea - hypopnea syndrome.
    
    ### 8. Loop Processing
    1. Slide the time window forward (e.g., slide 1 minute, adjustable according to actual needs), update the data range, and repeat steps 3 - 8 to continuously calculate the AHI value throughout the sleep monitoring period.
    
    Please note that this section pertains to the patented invention "Sleep Apnoea Event Detection Method" (ZL 202110167676.X), which was granted to the Chinese PLA General Hospital on 4 July, 2023. 
    """
    return ahi

def StageClassifier():
    """
    Pseudocode for StageClassifier:
	Input:
	    ecg: An array of electrocardiogram signals
	    rsp: An array of respiratory signals 
	    
	Output:
	    predict_result: NREM1(Non-Rapid Eye Movement1 sleep periods); NREM2_per,(Non-Rapid Eye Movement2 sleep periods) ; NREM3_per(Non-Rapid Eye Movement3 sleep periods); REM_per, (Rapid Eye Movement sleep periods)

	Procedure:    
    ### 1. RR Interval Calculation and Processing
    1. **Calculate RR Interval Sequence**
       - Calculate the time interval between adjacent R-waves based on the detected R-wave peaks to obtain the RR interval sequence (RRn).
    2. **Data Cleaning and Interpolation**
       - Check for abnormal values in the RR interval sequence (such as RR intervals that are too long or too short and outside normal physiological range). Replace the abnormal values with the average of adjacent RR intervals.
    
    ### 2. Feature Extraction
    1. **Extract Features from RR Interval Signal **
       - Calculate the mean of RR intervals (Mean_RR),the standard deviation of RR intervals (SDNN),the root mean square of the differences between adjacent RR intervals (RMSSD),the percentage of adjacent RR intervals with a difference greater than 50ms (pNN50),frequency domain features such as low frequency (LF), medium frequency (MF), high frequency (HF), and total power (TF) (by performing power spectral analysis on the RR interval sequence, using the Fast Fourier Transform (FFT)), and calculate their normalized values (such as LFn, MFn, HFn, etc.) and the ratios between different frequency components (such as LF/HF, etc.)
    2. **Extract Features from Respiratory Signal**
       - Calculate the inhalation time, exhalation time, and their related statistics (such as mean, standard deviation, etc.) of the breathing signal
       - Calculate the peak and trough related features of the breathing signal, such as the median of the time difference between the peak and the trough, the median of the inhalation area, the median of the exhalation area, and the median of the single breath area
       - Calculate the power spectral density of the breathing signal to obtain features such as the energy of high frequency, low frequency, very low frequency bands, and the ratio of high frequency to total frequency
        
    3. **Extract Features through Cardiorespiratory Coupling**
       - Calculate the cardiorespiratory coupling (CPC) index: Calculate the CPC index through a specific algorithm (such as based on the synchronous analysis of ECG and breathing signals), and obtain features such as the ratio of the sum of the CPC index in different frequency bands to the sum of the CPC index in the entire frequency band (such as high frequency/total power, low frequency/total power, very low frequency/total power, low frequency/high frequency, etc.)
       - Calculate the statistical features related to the CPC index, such as the standard deviation of the absolute value of the difference between the left and right halves
       
    ### 3. Sleep Stage Prediction
    1. **Input Features into the Model** (an existing trained BLSTM model.The accuracy and reliability of the model has been verified.(Wang, Z., et al. Development and Validation of Algorithms for Sleep Stage Classification and Sleep Apnea/Hypopnea Event Detection Using a Medical-Grade Wearable Physiological Monitoring System. in International Conference on Wireless Mobile Communication and Healthcare. 2021. Springer.))
       - Compose the extracted features into a feature vector and input it into the trained BLSTM model in the format required by the model.
    2. **Model Prediction**
       - The model predicts based on the input vector and outputs the probability values of each sleep stage category (such as NREM1, NREM2_per, NREM3_per, REM_per) corresponding to each time point.
    3. **Determine the Sleep Stage Result**
       - For each time point, select the category with the highest probability as the sleep stage result based on the output probability values of the model. For example, if the model predicts the sleep stage category probabilities at a certain time point as [0.1, 0.3, 0.4, 0.2] (corresponding to NREM1, NREM2_per, NREM3_per, REM_per respectively), then the sleep stage result at this time point is NREM3_per.
	"""
    return Sleep_Stage_result

def stat_sleep(stages):
    sl_re = dict()
    wake_time = np.sum(stages == 1)
    light_time = np.sum(stages == 2)
    deep_time = np.sum(stages == 3)
    rem_time = np.sum(stages == 4)
    sl_re['tol_len'] = len(stages)*30 / 3600
    sl_re['wake_per'] = wake_time / len(stages)*100
    sl_re['light_per'] = light_time / len(stages) * 100
    sl_re['deep_per'] = deep_time / len(stages) * 100
    sl_re['rem_per'] = rem_time / len(stages) * 100
    return sl_re

def sleep_process(ecg, rsp_ch, rsp_ab, spo, acc):
    rsp = rsp_ch + rsp_ab
    sleep_stage = StageClassifier(ecg, rsp)
    stages = sleep_stage
    slp_features = stat_sleep(stages)
    
    turnover_index = []
    lie_list = [2, 3, 4, 5]
    for current_pos in np.arange(len(acc.posture) - 2):
        if acc.posture[current_pos] in lie_list:
            if acc.posture[current_pos + 1] != acc.posture[current_pos] and (
                    acc.posture[current_pos + 1] in lie_list):
                turnover_index.append(current_pos)
    turnover_index = np.array(turnover_index)
    diff_turnover = np.diff(turnover_index)
    turnover_events_number = len(diff_turnover[diff_turnover >= 60])
    slp_features['turn_over'] = turnover_events_number
    
    ahi, evs = SleepApneaDetector(rsp_ch, rsp_ab, spo, acc)
    slp_features['AHI'] = ahi
    
    return slp_features

# smooth:
ecg_smoothed = smooth_signal(ecg_raw, size=200)
rsp_ch_smoothed = smooth_signal(rsp_ch_raw, size=25)
rsp_ab_smoothed = smooth_signal(rsp_ab_raw, size=25)
spo_smoothed = smooth_signal(spo_raw, size=10)
acc_smoothed = smooth_signal(acc_raw, size=10)

#Removing outliers
ecg_remove = remove_outliers(ecg_smoothed, size=200)
rsp_ch_remove = remove_outliers(rsp_ch_smoothed, size=25)
rsp_ab_remove = remove_outliers(rsp_ab_smoothed, size=25)
spo_remove = remove_outliers(spo_smoothed, size=10)
acc_remove = remove_outliers(acc_smoothed, size=10)

_, results = nk.ecg_peaks(ecg_remove, sampling_rate=200, method='Hamilton')
nn = results['ECG_R_Peaks']

# HRV features
hrv_feature = hrv.extract_features.get_time_domain_features(nn_intervals=nn)
hrv_feature2 = hrv.extract_features.get_frequency_domain_features(nn_intervals)

# Characterization of the arrhythmic load
ar_burden = rr_process(ecg_remove,nn)

# Respiratory characteristics
rsp_remove = rsp_ch_remove + rsp_ab_remove
info = {"RSP_Peaks": peaks, "RSP_Troughs": troughs}
rsp_info = nk.rsp_findpeaks(rsp_cleaned=rsp_remove, sampling_rate=25, method="khodadad2018")
rsp_feature = rsp_process(rsp_info,rsp_remove)

# Oxygen Characteristics
spo_feature = spo_process(spo_remove)

# Sleep features
slp_features = sleep_process(ecg_remove, rsp_ch_remove, rsp_ab_remove, spo_remove, acc_remove)

# Merge
Physiology_features = hrv_feature.merge(hrv_feature2, on='key', how='inner') \
    .merge(ar_burden, on='key', how='inner') \
    .merge(rsp_feature, on='key', how='inner') \
    .merge(spo_feature, on='key', how='inner') \
    .merge(slp_features, on='key', how='inner')

# clinical
clinical_features = pd.read_csv('clinical_data.csv')

all_features = pd.merge(Physiology_features, clinical_features, on='key', how='inner')

with pd.ExcelWriter('Physiological and Clinical Characteristics of Heart valve surgery patients.xlsx') as writer:
    clinical_features.to_excel(writer, sheet_name='cln', index=False)
    Physiology_features.to_excel(writer, sheet_name='phy', index=False)
    all_features.to_excel(writer, sheet_name='both', index=False)

In [ ]:
# Feature screening

excel_file_path = 'Physiological and Clinical Characteristics of Heart valve surgery patients.xlsx'


#Choose a different learner
models_dict = {
    "XGBoost": XGBClassifier(learning_rate=0.01, n_estimators=10, max_depth=1, random_state=1),
    "Logistic Regression": LogisticRegression(C=0.01, random_state=1),
    "Random Forest": RandomForestClassifier(n_estimators=10, max_depth=1, random_state=1),
    "SVM": SVC(kernel='linear', random_state=1, probability=True),
    "KNN": KNeighborsClassifier()
}

sheets = ['cln', 'phy', 'both']
models = ['XGBoost', 'Logistic Regression', 'Random Forest', 'SVM', 'KNN']

#Stored in the corresponding model sheet
with pd.ExcelWriter('Feature screened _data.xlsx') as writer:
  
    for model_name in models:
        estimator = models_dict[model_name]  
        

        for sheet in sheets:
            print(f"Processing data for model , sheet {sheet}")

            df = pd.read_excel(excel_file_path, sheet_name=sheet)
            X = df.iloc[:, :-1].values 
            y = df.iloc[:, -1].values  

            rfecv = RFECV(estimator=estimator, step=1, cv=StratifiedKFold(5), scoring='roc_auc')
            rfecv.fit(X, y)

            print(f"Optimal number of features for {model_name} on {sheet}: {rfecv.n_features_}")
            print(f"Ranking of features for {model_name} on {sheet}: {rfecv.ranking_}")

            selected_features = df.columns[:-1][rfecv.support_]
            processed_data = df[selected_features]               
            processed_data['Label'] = y                         

            output_sheet_name = f"{model_name}_{sheet}"
            processed_data.to_excel(writer, sheet_name=output_sheet_name, index=False)

In [ ]:
# Model tuning

excel_file_path = 'Feature screened _data.xlsx'

# Define the model and corresponding sheet names
# cln for clinical data, phy for physiologic data, both for total data
sheet_mapping = {
    "XGBoost": ["xgb_cln", "xgb_phy", "xgb_both"],
    "Logistic Regression": ["lr_cln", "lr_phy", "lr_both"],
    "Random Forest": ["rf_cln", "rf_phy", "rf_both"],
    "SVM": ["svm_cln", "svm_phy", "svm_both"],
    "KNN": ["knn_cln", "knn_phy", "knn_both"]
}

# Define initial parameters for each model
models = {
    "XGBoost": xgb.XGBClassifier(learning_rate=0.01, n_estimators=10, max_depth=1, random_state=1),
    "Logistic Regression": LogisticRegression(C=0.01, random_state=1),
    "Random Forest": RandomForestClassifier(n_estimators=10, max_depth=1, random_state=1),
    "SVM": SVC(kernel='poly', C=0.01, random_state=1, probability=True),
    "KNN": KNeighborsClassifier()
}

# Define the parameter grid
param_grids = {
    "XGBoost": {
        'n_estimators': list(range(10, 101, 10)),
        'max_depth': list(range(1, 10)),
        'learning_rate': [0.01, 0.1, 0.2],
        'random_state' : list(range(1,50))
    },
    "Logistic Regression": {
        'C': [0.01, 0.1, 0.2, 0.5, 1, 10]
        'random_state' : list(range(1,50))
    },
    "Random Forest": {
        'n_estimators': [10, 40, 70, 100],
        'max_depth': [1, 2, 4, 6, 8]
        'random_state' : list(range(1,50))
    },
    "SVM": {
        'C': [0.01, 0.1, 0.2, 0.5, 1],
        'kernel': ['linear', 'poly', 'rbf']
        'random_state' : list(range(1,50))
    },
    "KNN": {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance']
    }
}

# Read and process the corresponding sheet for each model and perform parameter search, training and evaluation
for model_name, sheets in sheet_mapping.items():
    model = models[model_name]
    print(f"processing: {model_name}")
    

    for sheet in sheets:
        print(f" data: {sheet}")
        
        df = pd.read_excel(excel_file_path, sheet_name=sheet)
        all_features = df.values
        X = all_features[:, :-1] 
        y = all_features[:, -1]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        

        grid_search = GridSearchCV(estimator=model, param_grid=param_grids[model_name],
                                   scoring='roc_auc', cv=5, n_jobs=-1)
        

        grid_search.fit(X_train, y_train)

        print(f" Best parameters found: {grid_search.best_params_}")
        print(f" Best AUC score: {grid_search.best_score_}")
        

        best_model = grid_search.best_estimator_
        y_pred_proba = best_model.predict_proba(X_test)[:, 1]
        test_auc = roc_auc_score(y_test, y_pred_proba)
        print(f" Test AUC score:: {test_auc}")


In [ ]:
# Model training

models = {
    "XGBoost": xgb.XGBClassifier(learning_rate=0.1, n_estimators=32, max_depth=1, random_state=30),
    "Logistic Regression": LogisticRegression(C=0.2, random_state=1),
    "Random Forest": RandomForestClassifier(n_estimators=40, max_depth=2, random_state=1),
    "SVM": SVC(kernel='poly', C=0.2, random_state=10, probability=True),
    "KNN": KNeighborsClassifier(n_neighbors=5, weights='uniform')
}

sheet_mapping = {
    "XGBoost": ["xgb_cln", "xgb_phy", "xgb_both"],
    "Logistic Regression": ["lr_cln", "lr_phy", "lr_both"],
    "Random Forest": ["rf_cln", "rf_phy", "rf_both"],
    "SVM": ["svm_cln", "svm_phy", "svm_both"],
    "KNN": ["knn_cln", "knn_phy", "knn_both"]
}

results = {}
predictions = {model_name: [] for model_name in models.keys()}

for model_name, model in models.items():
    a_auc_per_sheet = []  # Store the AUC value for each sheet
    for sheet in sheet_mapping[model_name]:
        df = pd.read_excel('Feature screened _data.xlsx', sheet_name=sheet)
        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]

        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        #5-fold cross validation
        skf = StratifiedKFold(n_splits=5, random_state=10, shuffle=True) 
        a_auc, a_acc, a_f1, a_pre = [], [], [], []
        sheet_predictions = []

        for tr_idx, test_idx in skf.split(X, y):
            lab = y
            model.fit(X[tr_idx], y[tr_idx])
            y_predict = model.predict_proba(X[test_idx])[:, -1] 
            sheet_predictions.extend(y_predict)  
            aauc = roc_auc_score(y[test_idx], y_predict)
            f1_s = f1_score(lab[test_idx.tolist()], predict)
            prec = precision_score(lab[test_idx.tolist()], predict)
            aauc = roc_auc_score(lab[test_idx.tolist()], y_predict[:, -1])
            a_auc.append(aauc)
            a_acc.append(acc)
            a_f1.append(f1_s)
            a_pre.append(prec)
        mean_auc = np.mean(a_auc)
        a_auc_per_sheet.append(mean_auc)
        predictions[model_name].append(np.array(sheet_predictions))  
        results[model_name] = a_auc_per_sheet
        print(f "{model_name}:")
        
        #Calculation of auc,acc,f1
        print(round(np.mean(a_auc), 2), '±', round(np.std(a_auc, ddof=1), 2))
        print(round(np.mean(a_acc), 2), '±', round(np.std(a_acc, ddof=1), 2))
        print(round(np.mean(a_f1), 2), '±', round(np.std(a_f1, ddof=1), 2))
        print(round(np.mean(a_pre), 2), '±', round(np.std(a_pre, ddof=1), 2))